# Clifford+RZ

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install --platform=manylinux2014_x86_64 --only-binary=:all: pyqrack

In [2]:
import collections
import math
import os
import random

import numpy as np

import tensorcircuit as tc
import tensorcircuit.compiler.simple_compiler as tcsc

from qiskit import QuantumCircuit
from qiskit.compiler import transpile

from pyqrack import QrackSimulator

# If we disable OpenCL and set the max CPU qubits to max integer, we bypass qubit widths limits.
os.environ["QRACK_MAX_CPU_QB"]="-1"

In [3]:
width = 4
max_magic = 4

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [4]:
def rand_1qb(sim, q):
    # th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    # lm = random.uniform(0, 4 * math.pi)
    # sim.u(th, ph, lm, q)
    sim.u(0, ph, 0, q)

def random_circuit(width, circ):
    t_count = 0
    single_bit_gates = circ.h, circ.x, circ.y
    two_bit_gates = circ.cx, circ.cy, circ.cz, circ.swap, circ.iswap

    for n in range(3 * width):
        # Single bit gates
        for j in range(width):
            random.choice(single_bit_gates)(j)
            if (t_count < max_magic) and ((width * width * random.random() / max_magic) < 1):
                rand_1qb(circ, j)
                t_count += 1

        # Multi bit gates
        bit_set = [i for i in range(width)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            random.choice(two_bit_gates)(b1, b2)

    return circ

If we have Qiskit and numpy installed, we can convert to a Qiskit circuit. The gate count and qubit width might be higher, but the circuit is entirely Clifford group, except for a single terminal layer of non-Clifford gates, (followed by either post selection or a rudimentary error-correction routine).

In [5]:
orig_circ = random_circuit(width, QuantumCircuit(width))
qsim = QrackSimulator(width, isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(orig_circ, 0)
qsim.out_to_file("qrack_circuit.chp")
qsim.reset_all()
qsim = None

In [14]:
circ = QrackSimulator.file_to_qiskit_circuit("qrack_circuit.chp")

if width <= 8:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

        ┌───┐   ┌───┐┌───┐                    ┌───┐┌───┐   ┌───┐        ┌───┐»
q_0: ───┤ H ├───┤ S ├┤ X ├─────────────────■──┤ S ├┤ H ├───┤ S ├────────┤ X ├»
        ├───┤   ├───┤└─┬─┘          ┌───┐┌─┴─┐└───┘├───┤┌──┴───┴──┐     └─┬─┘»
q_1: ───┤ S ├───┤ H ├──┼────■───────┤ X ├┤ X ├──■──┤ Y ├┤ Unitary ├───────┼──»
        ├───┤   ├───┤  │    │       └─┬─┘└───┘┌─┴─┐├───┤└──┬───┬──┘┌───┐  │  »
q_2: ───┤ S ├───┤ H ├──■────┼─────────┼───────┤ X ├┤ S ├───┤ H ├───┤ X ├──┼──»
        ├───┤   └───┘     ┌─┴─┐┌───┐  │  ┌───┐└───┘└───┘   └───┘   └─┬─┘  │  »
q_3: ───┤ S ├─────────────┤ X ├┤ S ├──┼──┤ H ├───────────────────────■────┼──»
        ├───┤             └───┘└───┘  │  └───┘                            │  »
q_4: ───┤ H ├─────────────────────────┼───────────────────────────────────┼──»
     ┌──┴───┴──┐                      │                                   │  »
q_5: ┤ Unitary ├──────────────────────┼───────────────────────────────────┼──»
     └──┬───┬──┘                      │             

We can further optimize by post-selecting with tensor network software.

In [15]:
tc.set_backend("tensorflow")
tc.set_contractor("auto")
tc.set_dtype("complex64")

net = tc.Circuit.from_qiskit(circ)
for b in range(width, circ.width()):
    net.post_select(b, keep=0)
net = tcsc.simple_compile(net)[0]

We can sample with `tensorcircuit` at this point, or we can try another round of optimization.

In [16]:
# %%time
# print(net.sample(allow_state=True, batch=1024, format="count_dict_bin"))

We can convert back to a Qiskit circuit.

In [17]:
circ = net.to_qiskit()
if width <= 8:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

omit non unitary gate in tensorcircuit when transforming to qiskit: any
omit non unitary gate in tensorcircuit when transforming to qiskit: any


     ┌───┐┌───┐┌───┐                    ┌───┐┌───┐┌───┐     ┌───┐┌───┐     »
q_0: ┤ H ├┤ S ├┤ X ├─────────────────■──┤ S ├┤ H ├┤ S ├─────┤ X ├┤ X ├──■──»
     ├───┤├───┤└─┬─┘          ┌───┐┌─┴─┐└───┘├───┤└───┘     └─┬─┘└─┬─┘  │  »
q_1: ┤ S ├┤ H ├──┼────■───────┤ X ├┤ X ├──■──┤ Y ├────────────┼────┼────┼──»
     ├───┤├───┤  │    │       └─┬─┘└───┘┌─┴─┐├───┤┌───┐┌───┐  │    │    │  »
q_2: ┤ S ├┤ H ├──■────┼─────────┼───────┤ X ├┤ S ├┤ H ├┤ X ├──┼────■────┼──»
     ├───┤└───┘     ┌─┴─┐┌───┐  │  ┌───┐└───┘└───┘└───┘└─┬─┘  │       ┌─┴─┐»
q_3: ┤ S ├──────────┤ X ├┤ S ├──┼──┤ H ├─────────────────■────┼───────┤ X ├»
     ├───┤          └───┘└───┘  │  └───┘                      │       └───┘»
q_4: ┤ H ├──────────────────────┼─────────────────────────────┼────────────»
     └───┘                      │                             │            »
q_5: ───────────────────────────┼─────────────────────────────┼────────────»
     ┌───┐                      │                             │  ┌───┐     »

Some phase gates tend to occur at the beginning of the circuit: these can safely be ignored. In fact, we can trace out the evolution of separable single-qubit states from the beginning of the circuit until they become entangled, for more gate elimination. Deterministic control qubits can be eliminated, as well.

In [18]:
to_cut = []
state = circ.width() * [False]
basis = circ.width() * [False]
has_depth = circ.width() * [False]
for i in range(len(circ.data)):
    op = circ.data[i].operation
    qubits = circ.data[i].qubits
    first_qubit = circ.find_bit(qubits[0])[0]
    if has_depth[first_qubit]:
        continue
    if len(qubits) == 1:
        if op.name == "h":
            basis[first_qubit] = not basis[first_qubit]
        elif ((not basis[first_qubit]) and ((op.name == "z") or (op.name == "s") or (op.name == "sdg"))) or (basis[first_qubit] and (op.name == "x")):
            to_cut.append(i)
        elif ((not basis[first_qubit]) and ((op.name == "x") or (op.name == "y"))) or (basis[first_qubit] and ((op.name == "y") or (op.name == "z"))):
            state[first_qubit] = not state[first_qubit]
        else:
            has_depth[first_qubit] = True
    elif op.name == "swap":
        second_qubit = circ.find_bit(qubits[1])[0]
        has_depth[first_qubit], has_depth[second_qubit] = has_depth[second_qubit], has_depth[first_qubit]
        state[first_qubit], state[second_qubit] = state[second_qubit], state[first_qubit]
        basis[first_qubit], basis[second_qubit] = basis[second_qubit], basis[first_qubit]
        if (not has_depth[first_qubit]) and (not has_depth[second_qubit]) and (state[first_qubit] == state[second_qubit]) and (basis[first_qubit] == basis[second_qubit]):
            to_cut.append(i)
    elif not basis[first_qubit] and op.name == "cx":
        if state[first_qubit]:
            circ.data[i].operation.name = "x"
            del circ.data[i].qubits[0]
            i -= 1
        else:
            to_cut.append(i)
    else:
        qc = circ.find_bit(qubits[0])[0]
        qt = circ.find_bit(qubits[1])[0]
        has_depth[qc] = True
        has_depth[qt] = True

to_cut.reverse()
for i in to_cut:
    del circ.data[i]
    
if width <= 8:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

     ┌───┐┌───┐┌───┐          ┌───┐┌───┐┌───┐     ┌───┐┌───┐     ┌───┐     »
q_0: ┤ H ├┤ S ├┤ X ├───────■──┤ S ├┤ H ├┤ S ├─────┤ X ├┤ X ├──■──┤ Y ├─────»
     ├───┤└───┘└─┬─┘┌───┐┌─┴─┐└───┘├───┤└───┘     └─┬─┘└─┬─┘  │  └───┘     »
q_1: ┤ H ├──■────┼──┤ X ├┤ X ├──■──┤ Y ├────────────┼────┼────┼────────────»
     ├───┤  │    │  └─┬─┘└───┘┌─┴─┐├───┤┌───┐┌───┐  │    │    │  ┌───┐     »
q_2: ┤ H ├──┼────■────┼───────┤ X ├┤ S ├┤ H ├┤ X ├──┼────■────┼──┤ H ├─────»
     └───┘┌─┴─┐┌───┐  │  ┌───┐└───┘└───┘└───┘└─┬─┘  │       ┌─┴─┐├───┤┌───┐»
q_3: ─────┤ X ├┤ S ├──┼──┤ H ├─────────────────■────┼───────┤ X ├┤ S ├┤ H ├»
     ┌───┐└───┘└───┘  │  └───┘                      │       └───┘└───┘└───┘»
q_4: ┤ H ├────────────┼─────────────────────────────┼──────────────────────»
     └───┘            │                             │                      »
q_5: ─────────────────┼─────────────────────────────┼──────────────────────»
     ┌───┐            │                             │  ┌───┐               »

If any ancilla qubit is nothing but an injection gadget, we can re-inject it.

In [19]:
sqrt1_2 = 1 / math.sqrt(2)
passable_gates = ["unitary", "x", "y", "z", "s", "sdg"]

unitary_stub = ""
for inst in circ.data:
    if inst.operation.name != "unitary":
        continue
    unitary_stub = inst
    break

if unitary_stub != "":
    for i in range(width, circ.width()):
        non_clifford = np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]])
        to_cut = []
        for j in reversed(range(len(circ.data))):
            op = circ.data[j].operation
            qubits = circ.data[j].qubits
            id = circ.find_bit(qubits[0])[0]
            if id == i:
                unitary_stub.qubits = (qubits[0],)
                break
        for j in reversed(range(len(circ.data))):
            op = circ.data[j].operation
            qubits = circ.data[j].qubits

            id = circ.find_bit(qubits[0])[0]
            if (len(qubits) < 2) and (id == i):
                to_cut.append(j)
                if op.name == "unitary":
                    non_clifford = np.matmul(non_clifford, op.params[0])
                elif op.name == "h":
                    non_clifford = np.matmul(non_clifford, np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]]))
                elif op.name == "x":
                    non_clifford = np.matmul(non_clifford, np.array([[0, 1], [1, 0]]))
                elif op.name == "y":
                    non_clifford = np.matmul(non_clifford, np.array([[0, -1j], [1j, 0]]))
                elif op.name == "z":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, -1]]))
                elif op.name == "s":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, 1j]]))
                elif op.name == "sdg":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, -1j]]))
                else: 
                    print("Something went wrong!")

                continue

            if len(qubits) < 2:
                continue

            if (id == i) or not (op.name == "cx"):
                # non_clifford = np.matmul(np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]]), non_clifford)
                unitary_stub.operation.params = [np.asarray(non_clifford)]
                circ.data.insert(j + 1, unitary_stub)
                break

            if (circ.find_bit(qubits[1])[0] == i) and (op.name == "cx"):
                circ.data[j].operation.name = "unitary"
                circ.data[j].operation.params.append(non_clifford)
                circ.data[j].qubits = (qubits[0],)
        
        for i in to_cut:
            del circ.data[i]

if width <= 8:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

     ┌───┐┌───┐┌───┐          ┌───┐┌───┐┌───┐     ┌───┐┌───┐     ┌───┐     »
q_0: ┤ H ├┤ S ├┤ X ├───────■──┤ S ├┤ H ├┤ S ├─────┤ X ├┤ X ├──■──┤ Y ├─────»
     ├───┤└───┘└─┬─┘┌───┐┌─┴─┐└───┘├───┤└───┘     └─┬─┘└─┬─┘  │  └───┘     »
q_1: ┤ H ├──■────┼──┤ X ├┤ X ├──■──┤ Y ├────────────┼────┼────┼────────────»
     ├───┤  │    │  └─┬─┘└───┘┌─┴─┐├───┤┌───┐┌───┐  │    │    │  ┌───┐     »
q_2: ┤ H ├──┼────■────┼───────┤ X ├┤ S ├┤ H ├┤ X ├──┼────■────┼──┤ H ├─────»
     └───┘┌─┴─┐┌───┐  │  ┌───┐└───┘└───┘└───┘└─┬─┘  │       ┌─┴─┐├───┤┌───┐»
q_3: ─────┤ X ├┤ S ├──┼──┤ H ├─────────────────■────┼───────┤ X ├┤ S ├┤ H ├»
     ┌───┐└───┘└───┘  │  └───┘                      │       └───┘└───┘└───┘»
q_4: ┤ H ├────────────┼─────────────────────────────┼──────────────────────»
     └───┘            │                             │                      »
q_5: ─────────────────┼─────────────────────────────┼──────────────────────»
     ┌───┐            │                             │  ┌───┐               »

`QrackSimulator` can run a Qiskit circuit. (After the file is saved and loaded, separable ancillae qubits will be removed.)

In [20]:
# basis_gates=["rz", "h", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap", "iswap"]
# circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)

os.rename("qrack_circuit.chp", "qrack_circuit_original.chp")

qsim = QrackSimulator(circ.width(), isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(circ, 0)

qsim.out_to_file("qrack_circuit.chp")

lines = []
with open("qrack_circuit.chp","r",encoding="utf-8") as file:
    lines = file.readlines()
lines[0] = str(width) + "\n"
with open("qrack_circuit.chp", "w", encoding="utf-8") as file:
    file.writelines(lines)

We can also sample with Qrack, as opposed to tensor circuits.

In [21]:
lines = []
with open("qrack_circuit.chp","r",encoding="utf-8") as file:
    lines = file.readlines()
chp_qb_count = int(lines[1])
is_clifford = True
for i in range(2 + 2 * chp_qb_count, len(lines)):
    if lines[i] != "(1,0) (0,0) (0,0) (1,0)\n":
        is_clifford = False
        break

if is_clifford:
    qsim = QrackSimulator.in_from_file("qrack_circuit.chp")

    %time
    print(qsim.measure_shots(list(range(width)), 1024))

    # if (width + max_magic) <= 6:
    #     print("Validating...")
    #     qsim.run_qiskit_circuit(orig_circ.inverse())
    #     results = dict(collections.Counter(qsim.measure_shots(list(range(width)), 1024)))
    #     if results == { 0: 1024 }:
    #         print("Passed!")
    #     else:
    #         print("Failed: ", results)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
[11, 11, 5, 12, 0, 14, 14, 7, 3, 15, 14, 8, 6, 5, 8, 12, 6, 12, 5, 11, 2, 1, 1, 9, 13, 9, 8, 0, 10, 9, 8, 1, 1, 11, 4, 5, 1, 9, 13, 5, 5, 5, 12, 3, 3, 6, 2, 6, 3, 15, 7, 5, 5, 5, 5, 1, 6, 1, 14, 6, 1, 6, 6, 6, 6, 5, 6, 12, 6, 11, 14, 9, 6, 6, 5, 10, 0, 14, 3, 11, 9, 14, 6, 13, 8, 11, 12, 7, 0, 4, 7, 4, 13, 5, 4, 3, 5, 1, 13, 1, 0, 8, 5, 13, 5, 7, 9, 10, 15, 8, 9, 11, 13, 0, 12, 1, 14, 8, 4, 10, 5, 6, 13, 8, 8, 15, 15, 10, 12, 11, 1, 8, 4, 10, 5, 6, 6, 13, 12, 7, 5, 2, 13, 9, 15, 13, 14, 12, 7, 14, 10, 6, 5, 1, 9, 11, 5, 1, 0, 3, 0, 3, 0, 0, 8, 1, 8, 0, 12, 9, 10, 15, 7, 9, 3, 12, 6, 8, 2, 6, 14, 6, 13, 12, 8, 4, 11, 15, 4, 0, 3, 1, 7, 14, 1, 11, 7, 8, 0, 3, 13, 5, 4, 11, 2, 4, 2, 3, 13, 14, 7, 2, 6, 8, 4, 9, 10, 3, 15, 9, 0, 13,